In [1]:
# Install required packages (run once)
!pip install qiskit qiskit-aer qiskit-nature pyscf qiskit-algorithms --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00


In [2]:
# Imports
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimatorV2
from qiskit.circuit.library import n_local
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

In [3]:
# Step 1: Define H2 molecule
driver = PySCFDriver(atom='H 0 0 0; H 0 0 0.74', basis='sto3g')
driver_result = driver.run()
second_q_ops = driver_result.second_q_ops()
main_op = second_q_ops[0]

# Step 2: Map to qubits
mapper = JordanWignerMapper()
qubit_op = mapper.map(main_op)

# Step 3: Set up AerSimulator and BackendEstimatorV2
simulator = AerSimulator(method='statevector')
estimator = BackendEstimatorV2(backend=simulator)

# Step 4: Define ansatz and optimizer
num_qubits = qubit_op.num_qubits
ansatz = n_local(num_qubits=num_qubits, rotation_blocks='ry', entanglement_blocks='cz', reps=2)
optimizer = SLSQP(maxiter=100)

# Step 5: Run VQE
vqe = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)
vqe_result = vqe.compute_minimum_eigenvalue(qubit_op)

# Step 6: Run exact classical solver for comparison
exact_solver = NumPyMinimumEigensolver()
exact_result = exact_solver.compute_minimum_eigenvalue(qubit_op)

# Step 7: Print results
print(f"VQE Ground State Energy of H2: {vqe_result.eigenvalue.real:.6f} Hartree")
print(f"Exact Ground State Energy of H2: {exact_result.eigenvalue.real:.6f} Hartree")


VQE Ground State Energy of H2: -0.705581 Hartree
Exact Ground State Energy of H2: -1.852388 Hartree
